In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import OrderedDict

In [2]:
classifier_pairs = np.load('D:/DATA/VESSEL_DATA/classifier_pairs.npy', allow_pickle = True)

In [3]:
def normalize(torch_Tensor, min_val, max_val):
    for img in torch_Tensor:
        img[img < min_val] = min_val
        img[img > max_val] = max_val
    torch_Tensor -= min_val
    torch_Tensor /= (max_val - min_val)
    return torch_Tensor

In [4]:
class lungCT_clsfr_DATA(Dataset):
    def __init__(self, classifier_pairs, split_type, transforms = None):
        self.classifier_pairs = classifier_pairs
        self.split_type = split_type
        self.transforms = transforms
        self.val_split = 0.1
        self.num_split = int(self.val_split * self.classifier_pairs.shape[0])
        self.min_val = -1024
        self.max_val = 100
        
        np.random.shuffle(self.classifier_pairs)
            
        if self.split_type == 'val':
            self.X_val = torch.Tensor([i[0] for i in tqdm(self.classifier_pairs[:self.num_split])]).view(-1, 1, 512, 512)
            self.X_val = normalize(self.X_val, self.min_val, self.max_val)
            self.y_val = torch.Tensor([i[1] for i in tqdm(self.classifier_pairs[:self.num_split])])
        elif self.split_type == 'test':
            self.X_test = torch.Tensor([i[0] for i in tqdm(self.classifier_pairs[self.num_split:2*self.num_split])]).view(-1, 1, 512, 512)
            self.X_test = normalize(self.X_test, self.min_val, self.max_val)
            self.y_test = torch.Tensor([i[1] for i in tqdm(self.classifier_pairs[self.num_split:2*self.num_split])])  
        elif self.split_type == 'train':
            self.X_train = torch.Tensor([i[0] for i in tqdm(self.classifier_pairs[2*self.num_split:])]).view(-1, 1, 512, 512)
            self.X_train = normalize(self.X_train, self.min_val, self.max_val)
            self.y_train = torch.Tensor([i[1] for i in tqdm(self.classifier_pairs[2*self.num_split:])])
    
    def __len__(self):
        if self.split_type == 'train':
            return self.y_train.shape[0]
        elif self.split_type == 'val':
            return self.y_val.shape[0]
        elif self.split_type == 'test':
            return self.y_test.shape[0]
    
    def __getitem__(self, idx):
        if self.split_type == 'train':
            return self.X_train[idx], self.y_train[idx]
        elif self.split_type == 'val':
            return self.X_val[idx], self.y_val[idx]
        elif self.split_type == 'test':
            return self.X_test[idx], self.y_test[idx]

In [5]:
BATCH_SIZE = 32

train = lungCT_clsfr_DATA(classifier_pairs, split_type = 'train', transforms = None)
val = lungCT_clsfr_DATA(classifier_pairs, split_type = 'val', transforms = None)
test = lungCT_clsfr_DATA(classifier_pairs, split_type = 'test', transforms = None)

train_loader = DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test, batch_size = BATCH_SIZE, shuffle = True)

100%|██████████| 859/859 [00:00<00:00, 430712.15it/s]


In [6]:
class lungCT_clsfr(nn.Module):
    def __init__(self, in_channels):
        super(lungCT_clsfr, self).__init__()
        
        self.conv1 = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(in_channels, 16, (5, 5))),
            ('maxpool1', nn.MaxPool2d((2, 2), 2)),
            ('activation1', nn.ReLU(True))
        ])) #254
        
        self.conv2 = nn.Sequential(OrderedDict([
            ('conv2', nn.Conv2d(16, 32, (5, 5))),
            ('maxpool2', nn.MaxPool2d((2, 2), 2)),
            ('activation2', nn.ReLU(True))
        ])) #125
        
        self.conv3 = nn.Sequential(OrderedDict([
            ('conv3', nn.Conv2d(32, 64, (5, 5))),
            ('maxpool3', nn.MaxPool2d((2, 2), 2)),
            ('activation3', nn.ReLU(True))
        ])) #60
        
        self.linear = nn.Sequential(OrderedDict([
            ('fc1', nn.Linear(60*60*64, 512)),
            ('activation4', nn.ReLU(True)),
            ('fc2', nn.Linear(512, 3)),
            ('activation5', nn.Softmax(dim = -1))
        ]))
        
    def forward(self, X):
        X = self.conv1(X)
        X = self.conv2(X)
        X = self.conv3(X)
        X = X.view(-1, 60*60*64)
        X = self.linear(X)
        return X

In [7]:
device=torch.device('cuda:0')

In [8]:
model = lungCT_clsfr(1).to(device)

In [9]:
def onehot_categorical_crossentropy(output, targets):
    #Expects torch tensor inputs
    output1 = output/torch.sum(output)
    epsilon = 1e-7
    output2 = torch.clamp(output1, epsilon, 1. - epsilon)
    return - torch.sum(targets * torch.log(output2))

In [10]:
def validation(model, val_loader, criterion, device):
    '''
    Helper validation loss and accuracy function for use during model training

    Args:
        model: model object (ex: vgg16, resnet or custom model using nn.Module)
        val_loader: data loader for validation set
        criterion: loss function used for training
        device: cuda or cpu device

    Returns:
        test_loss: the loss during validation testing
        accuracy: the accuracy during validation testing
    '''

    test_loss = 0
    total = 0
    correct = 0
    for data in tqdm(val_loader):
        X, y = data[0].to(device), data[1].to(device)
        output = model(X)
        test_loss += criterion(output, y).item()

        for index, i in enumerate(output):
            if torch.argmax(i) == torch.argmax(y[index]):
                correct+=1
            total+=1

    accuracy = correct/total

    return test_loss, accuracy

In [11]:
optimizer = torch.optim.Adam(model.parameters())
criterion = onehot_categorical_crossentropy

In [12]:
epochs = 5
plot_train_loss = []
plot_val_loss = []
plot_train_acc = []
plot_val_acc = []
for epoch in range(epochs):
    model.train()
    total = 0
    correct = 0
    train_loss = 0
    for data in tqdm(train_loader):
        X, y = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        for index, i in enumerate(output):
            if torch.argmax(i) == torch.argmax(y[index]):
                correct+=1
            total+=1

    model.eval()

    with torch.no_grad():
        val_loss, val_acc = validation(model, val_loader, criterion, device = device)

    train_acc = correct/total

    print("[%s/%s]\tTraining Loss: %s\tTraining Accuracy: %s\tValidation Loss: %s\tValidation Accuracy:%s" 
          % (epoch, epochs, train_loss, train_acc, val_loss, val_acc))
    
    plot_train_loss.append(train_loss)
    plot_train_acc.append(train_acc)
    plot_val_loss.append(val_loss)
    plot_val_acc.append(val_acc)

    model.train()

plot_train_loss = np.array(plot_train_loss) 
plot_val_loss = np.array(plot_val_loss)
plot_train_acc = np.array(plot_train_acc)
plot_val_acc = np.array(plot_val_acc)

  0%|          | 1/215 [00:07<28:15,  7.92s/it]


RuntimeError: CUDA out of memory. Tried to allocate 246.00 MiB (GPU 0; 4.00 GiB total capacity; 2.65 GiB already allocated; 195.80 MiB free; 2.74 GiB reserved in total by PyTorch)